# Configuración Inicial
Este bloque configura las rutas, variables y estructuras necesarias para procesar los datos.


In [272]:
import os
import pandas as pd
import re

# Configuración general
YEAR = 2020
RAW_FOLDER = "../data/raw"
PROCESSED_FOLDER = "../data/processed"
FILE_PATH_XLSX = f"{RAW_FOLDER}/data_{YEAR}.xlsx"
FILE_PATH_XLSB = f"{RAW_FOLDER}/data_{YEAR}.xlsb"
OUTPUT_PATH = f"{PROCESSED_FOLDER}/{YEAR}/pluviometros-eda-{YEAR}.csv"


In [273]:

# Mapeo de columnas
COLUMN_MAPPING = {
    'Fecha': ['Fecha', 'Date', 'Día', 'Dia'],
    'Kiteni': ['Base Operativa Kiteni', 'Kiteni', 'Obrador Kiteni'],
    'PS#1': ['PS1', 'PS#1', 'PS-1'],
    'PS#2': ['PS2', 'PS#2','PS-2', 'Planta de Selección #2'],
    'PS#3': ['PS3', 'PS#3',  'PS-3', 'Planta de Selección #3'],
    'PS#4': ['PS-4', 'PS#4/XV10007']
}


# Funciones Auxiliares
Este bloque contiene las funciones necesarias para procesar los datos:
1. Detección de encabezados.
2. Identificación de columnas relevantes.
3. Manejo de valores faltantes (`NaNs`).
4. Agregar información sobre rupturas.
5. Identificar outliers.


In [274]:
def detect_header_row(df, max_rows=15):
    """Detecta la fila que contiene los encabezados reales."""
    for row in range(max_rows):
        possible_header = df.iloc[row].values
        if any(isinstance(val, str) and len(val.strip()) > 0 for val in possible_header):
            return row
    return 0  # Si no se detecta, usar la primera fila

def detect_relevant_columns(df, column_mapping):
    """Detecta columnas relevantes basadas en nombres similares."""
    detected_columns = {}
    for expected_col, possible_names in column_mapping.items():
        for col in df.columns:
            if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                detected_columns[expected_col] = col
                break
    return detected_columns

def handle_missing_values(df, method='global_mean', monthly_averages=None):
    """Reemplaza NaNs con el promedio por columna (global o mensual)."""
    for col in df.columns:
        if col != 'Fecha':
            if method == 'global_mean':
                # Promedio global por columna
                mean_value = df[col].replace(0, pd.NA).mean(skipna=True)
                df[col] = df[col].fillna(mean_value)
            elif method == 'monthly_mean' and monthly_averages is not None:
                # Promedio mensual basado en datos disponibles
                df['Mes'] = pd.to_datetime(df['Fecha']).dt.month
                df[col] = df.apply(
                    lambda row: monthly_averages.loc[row['Mes'], col]
                    if pd.isnull(row[col]) else row[col], axis=1
                )
    return df


# Carga de Datos
Este bloque carga los datos desde el archivo Excel y consolida todas las hojas.


In [275]:
if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")

Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']


In [276]:
all_processed_data = pd.DataFrame()

if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")


Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']


In [277]:
for sheet_name in sheet_names:
    print(f"\nProcesando hoja: {sheet_name}")

    try:
        # Detección de encabezados
        sheet_preview = excel_data.parse(sheet_name, nrows=15)
        header_row = detect_header_row(sheet_preview)
        # Carga completa de datos
        sheet_data = excel_data.parse(sheet_name, skiprows=header_row)

        new_column_names = {}
        for expected_col, possible_names in COLUMN_MAPPING.items():
            for col in sheet_data.columns:
                if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                    new_column_names[col] = expected_col
                    break

        # Renombrar columnas según el mapeo
        sheet_data = sheet_data.rename(columns=new_column_names)

        columnas = sheet_data.columns.tolist()
        for col in columnas:
            if col not in COLUMN_MAPPING.keys():
                sheet_data = sheet_data.drop(columns=[(col)])

        sheet_data['Fecha'] = pd.to_datetime(sheet_data['Fecha'], errors='coerce')
        sheet_data['Mes'] = sheet_data['Fecha'].dt.month

        # Sustituir NaN por la media de cada mes
        numeric_columns = sheet_data.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_columns:
            sheet_data[col] = sheet_data.groupby('Mes')[col].transform(lambda x: x.fillna(x.mean()))

        # Concatenar datos procesados
        all_processed_data = pd.concat([all_processed_data, sheet_data], ignore_index=True)

    except Exception as e:
        print(f"Error procesando la hoja {sheet_name}: {e}")

print("\nDatos consolidados:")
display(all_processed_data.head())


Procesando hoja: ENERO

Procesando hoja: FEBRERO

Procesando hoja: MARZO

Procesando hoja: ABRIL

Procesando hoja: MAYO

Procesando hoja: JUNIO

Procesando hoja: JULIO

Procesando hoja: AGOSTO

Procesando hoja: SEPTIEMBRE

Procesando hoja: OCTUBRE

Procesando hoja: NOVIEMBRE

Procesando hoja: DICIEMBRE

Datos consolidados:


,Fecha,Kiteni,PS#1,PS#2,PS#3,PS#4,Mes
0,2020-01-01,5.0,73.2,9.2,1.0,2.0,1
1,2020-01-02,41.4,9.2,11.8,0.2,0.6,1
2,2020-01-03,2.8,32.0,10.6,0.9,0.8,1
3,2020-01-04,0.0,0.0,0.0,0.4,0.2,1
4,2020-01-05,13.6,4.4,24.2,0.2,0.4,1


In [278]:
# Mostrar resumen general de NaN por columna
# Mostrar cantidad total de NaN por columna
print("Cantidad total de valores NaN por columna:")
total_nan = all_processed_data.isnull().sum()
print(total_nan)

# Identificar columnas con valores NaN y su tipo de dato
nan_columns = total_nan[total_nan > 0].index  # Columnas con al menos un NaN
for col in nan_columns:
    print(f"Columna: {col}, Tipo: {all_processed_data[col].dtype}, Total NaN: {total_nan[col]}")

# Excluir la columna de agrupamiento ('Mes') y calcular NaN por mes y por columna
nan_analysis = (
    all_processed_data.drop(columns=['Mes'])  # Excluye la columna 'Mes' si existe
    .groupby(all_processed_data['Mes'])       # Agrupa usando la columna 'Mes'
    .apply(lambda x: x.isnull().sum())        # Calcula NaN por columna
)



Cantidad total de valores NaN por columna:
Fecha     0
Kiteni    0
PS#1      0
PS#2      0
PS#3      0
PS#4      0
Mes       0
dtype: int64


# Exportar Resultados
Guarda los datos procesados en un archivo CSV consolidado.


In [279]:
all_processed_data.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"Datos consolidados guardados en: {OUTPUT_PATH}")


Datos consolidados guardados en: ../data/processed/2020/pluviometros-eda-2020.csv


# Graficos mensuales


In [280]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Crear un archivo PDF
OUTPUT_PATH = f"{PROCESSED_FOLDER}/{YEAR}/graphic-pluviometros-eda-{YEAR}.pdf"
with PdfPages(OUTPUT_PATH) as pdf:
    # Iterar sobre cada mes
    for mes in range(1, 13):
        # Filtrar los datos por mes
        data_mes = all_processed_data[pd.to_datetime(all_processed_data['Fecha']).dt.month == mes]
        promedios_mes = data_mes.mean(numeric_only=True)

        # Crear el gráfico
        plt.figure(figsize=(10, 6))
        promedios_mes.plot(kind='bar')
        plt.title(f"Promedio de Lluvia - Mes {mes}")
        plt.xlabel("Sensores/Tramos")
        plt.ylabel("Promedio de Lluvia")
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Guardar el gráfico en el PDF
        pdf.savefig()  # Guarda la figura actual en el PDF
        plt.close()  # Cierra la figura para no sobrecargar la memoria

print(f"Gráficos mensuales guardados en {OUTPUT_PATH}")


Gráficos mensuales guardados en ../data/processed/2020/graphic-pluviometros-eda-2020.pdf
